In [64]:
! pip install mysql-connector-python


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Importing Libraries

In [65]:
import pandas as pd
import sqlite3
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

### Reading data from a CSV file into a dataframe
Dataset Used -  AMAZON SALES/REVIEWS from Kaggle


In [66]:
name = "sales.db"
table_name = "sales_data_7"
df = pd.read_csv('/Users/vardasinghal/Downloads/SQL Projects/amazon.csv')



## Data Preparation

Changed string to numeric for actual_price , discounted_price , discount_percentage, rating_count, rating



Extracted item as product_bought and top_category from category for analysis

Extracted brand from product name 

In [67]:
# Data Preparation
#Changing string to numneric
df['actual_price'] = df['actual_price'].str.replace('₹','').str.replace(',', '').astype(float)
df['discounted_price'] = df['discounted_price'].str.replace('₹','').str.replace(',', '').astype(float)
df['discount_percentage'] = df['discount_percentage'].str.replace('%', '').astype(float)
df['rating_count'] = df['rating_count'].str.replace(',', '').astype(float)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

#Retrieving subcategory
# Split the 'category' column by '|' and get the last element of each split
df['product_bought'] = df['category'].str.split('|').str[-1]
df['top_category'] = df['category'].str.split('|').str[0]

#Retrieve Brand
df['brand'] = df['product_name'].str.split(' ').str[0]



## Connecting to the database

In [ ]:
#connect to a database so that I can use SQL
conn = sqlite3.connect(name) #if the db does not exist, this creates a Any_Database_Name.db file in the current directory
#store your table in the database:
df.to_sql(table_name, conn)

# Analysis

In [ ]:
#Overview of data
df

## Analysis by Category

In [ ]:
#1) Overview of Category
#2) Top Category by Volume

In [ ]:
#Category Split
get_category = 'SELECT DISTINCT CATEGORY FROM sales_data_7'
category_df = pd.read_sql(get_category, conn)
category_df

In [ ]:
#Top Category by Volume
get_category_count = '''
SELECT category, 
COUNT(*) as category_count
from sales_data_7
group by 1
ORDER BY 2 DESC
'''
category_count_df = pd.read_sql(get_category_count, conn)
category_count_df 

Thought Process:
To focus on the final product purchased, I extracted the last item from the category hierarchy, as it contains multiple subcategories. (See Data Preparation for details.

## Analysis by Product_Bought

In [ ]:
#What will it answer
#1) Product Breakdown by Volume and Revenue
#2) Products with Above-Average Sales

In [ ]:
#Product bought split by volume
get_product_count = '''
SELECT product_bought , COUNT(*) as product_volume
from sales_data_7
GROUP BY 1
ORDER BY 2 DESC
'''

product_count_df = pd.read_sql(get_product_count, conn)
product_count_df 

In [ ]:
#Top Products by Revenue
get_product_count_revenue = '''
SELECT product_bought, 
SUM(discounted_price) as total_sales
from sales_data_7
group by 1
ORDER BY 2 DESC
'''
product_count_revenue_df = pd.read_sql(get_product_count_revenue, conn)
product_count_revenue_df 

In [ ]:
#Filtering products where average price per product is greater than total average sales
get_product_average_revenue = '''
with cte as
(
SELECT product_bought, 
AVG(discounted_price) as avg_product_price
from sales_data_7
group by 1
)
SELECT product_bought , avg_product_price
FROM CTE
WHERE avg_product_price > (SELECT AVG(discounted_price) FROM sales_data_7 )
order by 2 desc
'''
product_average_df = pd.read_sql(get_product_average_revenue, conn)
product_average_df

Thoughts:
I wanted to analyze brand performance across product categories, so I extracted the brand from the product name, as the same brands appear in multiple categories

In [ ]:
# What will it answer
#1) Top brands by product category 
#2) Average Brand Rating by Category (with High Sales)
#3) Most Popular Brand
#4) Brand Loyalty
#5) Brand Competition (by Rating) but having similar price

In [ ]:
#Top Selling Brand for product_bought(category) based on total revenue
get_brand = '''
with cte as
(
select product_bought , brand , product_bought, SUM(discounted_price) as total_sales,
 DENSE_RANK() OVER (PARTITION BY product_bought ORDER BY SUM(discounted_price) desc) as rnk
 FROM sales_data_7
 group by 1 , 2
)
select product_bought, brand , total_sales
from cte 
where rnk <= 1
ORDER BY 3 DESC
'''
get_brand_by_topproduct = pd.read_sql(get_brand, conn)
get_brand_by_topproduct

In [ ]:
#Average rating for brands where sales(discounted_price) > 50000 across all product_bought(category) and rating frequency > 3
get_brand_rating = '''
SELECT product_bought,
       brand, 
       AVG(rating) AS avg_rating, 
       COUNT(*) AS frequency
FROM (
    SELECT product_bought,
           brand,
           rating,
           discounted_price,
           SUM(discounted_price) OVER (PARTITION BY product_bought) AS total_sales
    FROM sales_data_7
) AS subquery
WHERE total_sales > 50000
GROUP BY product_bought, brand
HAVING COUNT(*) >= 3
ORDER BY product_bought,  frequency DESC , avg_rating DESC;
'''
get_brand_rating = pd.read_sql(get_brand_rating, conn)
get_brand_rating

In [ ]:
 #Most Popular Brand and its category by Review Count
get_popular_product = '''
SELECT 
    brand, product_bought,
    COUNT(*) AS review_count
FROM sales_data_7
GROUP BY brand , 2
ORDER BY review_count DESC;
'''
get_popular_product = pd.read_sql(get_popular_product, conn)
get_popular_product

In [ ]:
#Brand Loyalty showing multiple purchase of same brand per user
get_brand_loyal = '''
SELECT brand , distinct_products_byuser
from
(
SELECT brand, user_id, COUNT(DISTINCT product_id) AS distinct_products_byuser
FROM sales_DATA_7
GROUP BY brand, user_id
HAVING COUNT(DISTINCT product_id) > 1
ORDER BY  distinct_products_byuser DESC
) as subquery
'''
get_brand_loyal = pd.read_sql(get_brand_loyal, conn)
get_brand_loyal

In [ ]:
#Brands with similar price across product_bought and their rating
get_brands_similar_price = '''
SELECT DISTINCT
    a.product_bought, 
    a.brand AS brand_1, 
    a.discounted_price AS discounted_price_1, 
    a.rating AS rating_1, 
    b.brand AS brand_2, 
    b.discounted_price AS discounted_price_2, 
    b.rating AS rating_2
FROM 
    sales_data_7 AS a
JOIN 
    sales_data_7 AS b
    ON a.product_bought = b.product_bought  -- Same product_bought category
    AND a.discounted_price = b.discounted_price  -- Same discounted price
    AND a.brand != b.brand  -- Different brands
ORDER BY 
    a.product_bought, a.discounted_price;
'''
get_brands_similar_price  = pd.read_sql(get_brands_similar_price , conn)
get_brands_similar_price 